# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

!conda install -c conda-forge gmaps

!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


Solving environment: ...working... done

# All requested packages already installed.



Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [56]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
input_file = "Output/city_weather_data.csv"
weather_data = pd.read_csv(input_file)
weather_data.head()

,city,lat,lng,max_temp,humidity,cloudiness,wind_speed,country,date
0,Ilulissat,69.2167,-51.1000,21.20,68,20,5.75,GL,1614940588
1,North Bend,43.4065,-124.2243,53.60,54,90,13.80,US,1614940467
2,Ondjiva,-17.0667,15.7333,87.80,30,20,5.75,AO,1614940588
3,East London,-33.0153,27.9116,75.20,60,20,16.11,ZA,1614940588
4,Krutikha,53.9599,81.2093,24.04,83,81,17.72,RU,1614940589


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [19]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [22]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_data[["lat", "lng"]].astype(float)
locations.head()

,lat,lng
0,69.2167,-51.1000
1,43.4065,-124.2243
2,-17.0667,15.7333
3,-33.0153,27.9116
4,53.9599,81.2093


In [25]:
# Fill NaN values and convert to float
humidity = weather_data["humidity"].astype(float)
humidity.head()

0    68.0
1    54.0
2    30.0
3    60.0
4    83.0
Name: humidity, dtype: float64

In [27]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [32]:
hotel_df=weather_data.loc[(weather_data['max_temp'] < 80) & (weather_data['max_temp'] > 70) & (weather_data['wind_speed'] < 10) & (weather_data['cloudiness'] == 0)]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
hotel_df['Hotel Name'] = ''
hotel_df.head()

<ipython-input-34-15d30b78ccb4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,city,lat,lng,max_temp,humidity,cloudiness,wind_speed,country,date,Hotel Name
63,Hermanus,-34.4187,19.2345,78.01,50,0,5.01,ZA,1614940597,
79,Road Town,18.4167,-64.6167,75.99,92,0,6.76,VG,1614940600,
112,Pochutla,15.7432,-96.4661,71.01,80,0,2.68,MX,1614940605,
233,Nicoya,10.1483,-85.4520,71.46,79,0,7.34,CR,1614940625,
262,Aserrío de Gariché,8.4833,-82.8000,71.10,73,0,2.82,PA,1614940630,


In [ ]:
for city in hotel_df['city']
    
    # geocoordinates
    target_coordinates = str(hotel_df['lat']) + ', ' str(hotel_df[lng])
    target_radius = 5000
    target_type = "hotel"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": gkey
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

In [43]:
lat_arr=[]

In [44]:
for lat, data in hotel_df['lat'].items():
    lat_arr.append(data)
    #print(data)
lat_arr

[-34.4187,
 18.4167,
 15.7432,
 10.1483,
 8.4833,
 -15.8631,
 -34.0033,
 -29.4667,
 16.8634]

In [45]:
lng_arr=[]

In [46]:
for lng, data in hotel_df['lng'].items():
    lng_arr.append(data)
    #print(data)
lng_arr

[19.2345,
 -64.6167,
 -96.4661,
 -85.45200000000001,
 -82.8,
 -38.8828,
 24.7314,
 149.85,
 -99.8901]

In [107]:
hotel_name=[]

In [108]:
x = range(len(lat_arr))
for n in x:
    #print(lat_arr[n])
    # geocoordinates
    target_coordinates = str(lat_arr[n]) + ', ' + str(lng_arr[n])
    #print(target_coordinates)
    target_radius = 5000
    target_keyword = "hotel"
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "keyword": target_keyword,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    places_data = response.json()
    #print(n)
    try:
        hotel_df.iloc[n,9]=places_data['results'][0]['name']
    except:
        hotel_df.iloc[n,9]='(None)'
#print(places_data)

C:\Users\ryanp\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [83]:
#places_data['results'][0]['name']

In [103]:
hotel_df.rename(columns = {'city':'City'}, inplace = True)
hotel_df.rename(columns = {'country':'Country'}, inplace = True)
hotel_df.rename(columns = {'lat':'Lat'}, inplace = True)
hotel_df.rename(columns = {'lng':'Lng'}, inplace = True)
hotel_df

C:\Users\ryanp\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,City,Lat,Lng,max_temp,humidity,cloudiness,wind_speed,Country,date,Hotel Name
63,Hermanus,-34.4187,19.2345,78.01,50,0,5.01,ZA,1614940597,Ocean Eleven
79,Road Town,18.4167,-64.6167,75.99,92,0,6.76,VG,1614940600,Wyndham Tortola BVI Lambert Beach Resort
112,Pochutla,15.7432,-96.4661,71.01,80,0,2.68,MX,1614940605,(None)
233,Nicoya,10.1483,-85.4520,71.46,79,0,7.34,CR,1614940625,Hotel Doña Marta
262,Aserrío de Gariché,8.4833,-82.8000,71.10,73,0,2.82,PA,1614940630,Los Higuerones
346,Belmonte,-15.8631,-38.8828,71.60,88,0,4.61,BR,1614940645,Pousada do Diogo
355,Kruisfontein,-34.0033,24.7314,79.00,53,0,8.99,ZA,1614940646,(None)
391,Moree,-29.4667,149.8500,75.20,31,0,6.91,AU,1614940532,Gwydir Thermal Pools Motel & Caravanpark
550,Acapulco de Juárez,16.8634,-99.8901,71.28,77,0,4.00,MX,1614940680,Hotel Costa Miramar


In [104]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [117]:
def merge(list1, list2): 
      
    merged_list = [(list1[i], list2[i]) for i in range(0, len(list1))] 
    return merged_list 
coordinates=merge(lat_arr,lng_arr)

In [118]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))